In [6]:

import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.



In [4]:
pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 631.7 kB/s eta 0:00:0031m84.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 300.7 kB/s eta 0:00:001m485.5 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 373.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 498.8 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 472.9 kB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 1.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 4.2 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 1.6 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 918.4/918.4 kB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 355.8 kB/s eta 0:00:00m eta 0:00:010:00:02
Note: you may need to restart the kernel to use updated packages.




### Below is a helper Function which generates random colors which can be used to give different colors to your plots.Feel free to use it


In [7]:


def random_colours(number_of_colors):
    '''
    Simple function for random colours generation.
    Input:
        number_of_colors - integer value indicating the number of colours which are going to be generated.
    Output:
        Color in the following format: ['#E86DA4'] .
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors



## Reading the Data

In [9]:
df = pd.read_csv("/home/dawit/10_Acadamy/Copy of cleaned_fintech_data.csv")
df

,Unnamed: 0,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp
0,0.0,Thu Jun 17 06:26:34 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Giving forth life is becoming a burden in Keny...,Giving forth life becoming burden Kenya This m...,"Sentiment(polarity=0.3194444444444445, subject...",0.3194444444444445,0.5305555555555556,en,0,...,reen_law,398,70,223,NaN,NaN,janetmachuka_,NaN,NaN,2021-06-17 06:26:34+00:00
1,1.0,Thu Jun 17 06:26:37 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Teenmaar - 26cr\nPanja - 32.5cr\nGabbarsingh -...,Teenmaar crPanja crGabbarsingh cr Khaleja Kuda...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,in,0,...,Amigo9999_,19047,132,1084,NaN,NaN,maheshblood,NaN,India,2021-06-17 06:26:37+00:00
2,2.0,Thu Jun 17 06:26:42 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Rei chintu 2013 lo Vachina Ad Nizam ne 2018 lo...,Rei chintu lo Vachina Ad Nizam ne lo kottaru f...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,hi,0,...,MallaSuhaas,47341,2696,2525,NaN,NaN,Hail_Kalyan,NaN,Vizag,2021-06-17 06:26:42+00:00
3,3.0,Thu Jun 17 06:26:44 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Today is World Day to Combat #Desertification ...,Today World Day Combat Restoring degraded land...,"Sentiment(polarity=0.25, subjectivity=0.65)",0.25,0.65,en,0,...,CIACOceania,7039,343,387,NaN,"Desertification, Drought, resilience",EdwardVrkic,NaN,Papua New Guinea,2021-06-17 06:26:44+00:00
4,4.0,Thu Jun 17 06:26:47 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Hearing #GregHunt say he's confident vaccines ...,Hearing say 's confident vaccines delivered li...,"Sentiment(polarity=0.5, subjectivity=0.8333333...",0.5,0.8333333333333334,en,0,...,MccarronWendy,26064,419,878,NaN,"GregHunt, Morrison",WriteWithDave,NaN,"Sydney, New South Wales",2021-06-17 06:26:47+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5616,2805.0,Thu Jun 17 10:21:02 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Lambie has voted AGAINST Medivac, Carbon tax, ...",Lambie voted AGAINST Medivac Carbon tax protec...,"Sentiment(polarity=0.13636363636363635, subjec...",0.13636363636363635,0.45454545454545453,en,0,...,TrixAnabella,65144,644,1064,NaN,NaN,davidbewart,NaN,NaN,2021-06-17 10:21:02+00:00
5617,2806.0,Thu Jun 17 10:21:03 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Idhi me bathuku \nMost Day1 Records In Nizam f...,Idhi bathuku Most Day1 Records In Nizam Non BB...,"Sentiment(polarity=0.5, subjectivity=0.5)",0.5,0.5,in,0,...,Faninexile1,4796,713,227,NaN,NaN,Maheshtarakian1,NaN,NaN,2021-06-17 10:21:03+00:00
5618,2807.0,Thu Jun 17 10:21:08 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",welkin moon giveaway bc i finally have money!\...,welkin moon giveaway bc finally money -u got t...,"Sentiment(polarity=0.15416666666666667, subjec...",0.15416666666666667,0.5,en,0,...,acquitaine_,520,365,581,NaN,NaN,siilveon,NaN,deez nuts,2021-06-17 10:21:08+00:00
5619,2808.0,Thu Jun 17 10:21:09 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Manam edi chesina \n\nDaaniki kuda elevation i...,Manam edi chesina Daaniki kuda elevation istad...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,et,0,...,MbGodavari,17010,770,557,NaN,NaN,KonaseemaSSMBFC,NaN,NaN,2021-06-17 10:21:09+00:00


In [10]:
print(df.shape)

(5621, 21)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5621 entries, 0 to 5620
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5620 non-null   float64
 1   created_at              5621 non-null   object 
 2   source                  5621 non-null   object 
 3   original_text           5621 non-null   object 
 4   clean_text              5617 non-null   object 
 5   sentiment               5621 non-null   object 
 6   polarity                5621 non-null   object 
 7   subjectivity            5621 non-null   object 
 8   lang                    5621 non-null   object 
 9   favorite_count          5621 non-null   object 
 10  retweet_count           5621 non-null   object 
 11  original_author         5621 non-null   object 
 12  screen_count            5621 non-null   object 
 13  followers_count         5621 non-null   object 
 14  friends_count           5621 non-null   

## filling misisng value using mode since non-numeric 

In [14]:
df['hashtags'].fillna(df['hashtags'].mode()[0], inplace=True)
df['place'].fillna(df['place'].mode()[0], inplace=True)
df['place_coord_boundaries'].fillna(df['place_coord_boundaries'].mode()[0], inplace=True)
df['clean_text'].fillna(df['clean_text'].mode()[0], inplace=True)
df['possibly_sensitive'].fillna(df['possibly_sensitive'].mode()[0], inplace=True)

In [15]:
# Assuming 'column_name_to_remove' is the name of the unnecessary column
df.drop('Unnamed: 0', axis=1, inplace=True)

# EDA

In [17]:
df.head()

,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp
0,Thu Jun 17 06:26:34 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Giving forth life is becoming a burden in Keny...,Giving forth life becoming burden Kenya This m...,"Sentiment(polarity=0.3194444444444445, subject...",0.3194444444444445,0.5305555555555556,en,0,0,reen_law,398,70,223,False,Prabhas,janetmachuka_,place,"Hyderabad, India",2021-06-17 06:26:34+00:00
1,Thu Jun 17 06:26:37 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Teenmaar - 26cr\nPanja - 32.5cr\nGabbarsingh -...,Teenmaar crPanja crGabbarsingh cr Khaleja Kuda...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,in,0,0,Amigo9999_,19047,132,1084,False,Prabhas,maheshblood,place,India,2021-06-17 06:26:37+00:00
2,Thu Jun 17 06:26:42 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Rei chintu 2013 lo Vachina Ad Nizam ne 2018 lo...,Rei chintu lo Vachina Ad Nizam ne lo kottaru f...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,hi,0,0,MallaSuhaas,47341,2696,2525,False,Prabhas,Hail_Kalyan,place,Vizag,2021-06-17 06:26:42+00:00
3,Thu Jun 17 06:26:44 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Today is World Day to Combat #Desertification ...,Today World Day Combat Restoring degraded land...,"Sentiment(polarity=0.25, subjectivity=0.65)",0.25,0.65,en,0,0,CIACOceania,7039,343,387,False,"Desertification, Drought, resilience",EdwardVrkic,place,Papua New Guinea,2021-06-17 06:26:44+00:00
4,Thu Jun 17 06:26:47 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Hearing #GregHunt say he's confident vaccines ...,Hearing say 's confident vaccines delivered li...,"Sentiment(polarity=0.5, subjectivity=0.8333333...",0.5,0.8333333333333334,en,0,0,MccarronWendy,26064,419,878,False,"GregHunt, Morrison",WriteWithDave,place,"Sydney, New South Wales",2021-06-17 06:26:47+00:00


In [18]:
df.describe()

,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp
count,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621,5621
unique,2811,43,855,801,455,246,256,19,2,2,2340,2712,1482,1478,3,171,749,1,1112,2811
top,Thu Jun 17 06:26:34 +0000 2021,"<a href=""http://twitter.com/download/android"" ...","""I ask again - who pays?""\n\nAndrew Neil chall...",`` I ask pays `` Andrew Neil challenges Rishi ...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,0,SustainTrends,331,5928,159,False,Prabhas,TWTM__,place,"Hyderabad, India",2021-06-17 06:26:34+00:00
freq,2,3322,182,186,1780,2134,1846,3408,5620,5620,46,10,36,48,5618,5010,234,5621,2128,2


In [25]:
import pyLDAvis.gensim_models as gensimvis
import pickle
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()



In [19]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [20]:
df.head()

,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp
0,Thu Jun 17 06:26:34 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Giving forth life is becoming a burden in Keny...,Giving forth life becoming burden Kenya This m...,"Sentiment(polarity=0.3194444444444445, subject...",0.3194444444444445,0.5305555555555556,en,0,0,reen_law,398,70,223,False,Prabhas,janetmachuka_,place,"Hyderabad, India",2021-06-17 06:26:34+00:00
1,Thu Jun 17 06:26:37 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Teenmaar - 26cr\nPanja - 32.5cr\nGabbarsingh -...,Teenmaar crPanja crGabbarsingh cr Khaleja Kuda...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,in,0,0,Amigo9999_,19047,132,1084,False,Prabhas,maheshblood,place,India,2021-06-17 06:26:37+00:00
2,Thu Jun 17 06:26:42 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Rei chintu 2013 lo Vachina Ad Nizam ne 2018 lo...,Rei chintu lo Vachina Ad Nizam ne lo kottaru f...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,hi,0,0,MallaSuhaas,47341,2696,2525,False,Prabhas,Hail_Kalyan,place,Vizag,2021-06-17 06:26:42+00:00
3,Thu Jun 17 06:26:44 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Today is World Day to Combat #Desertification ...,Today World Day Combat Restoring degraded land...,"Sentiment(polarity=0.25, subjectivity=0.65)",0.25,0.65,en,0,0,CIACOceania,7039,343,387,False,"Desertification, Drought, resilience",EdwardVrkic,place,Papua New Guinea,2021-06-17 06:26:44+00:00
4,Thu Jun 17 06:26:47 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Hearing #GregHunt say he's confident vaccines ...,Hearing say 's confident vaccines delivered li...,"Sentiment(polarity=0.5, subjectivity=0.8333333...",0.5,0.8333333333333334,en,0,0,MccarronWendy,26064,419,878,False,"GregHunt, Morrison",WriteWithDave,place,"Sydney, New South Wales",2021-06-17 06:26:47+00:00


In [21]:
def label_sentiments_in_dataframe(df, sentiment_column='sentiment'):
    def label_sentiment(row):
        # Extracting polarity and subjectivity from the string
        parts = row[sentiment_column].split(',')
        polarity = float(parts[0].split('polarity=')[1])
        subjectivity = float(parts[1].split('subjectivity=')[1].rstrip(')'))

        # Detailed sentiment classification
        if polarity > 0.5 and subjectivity > 0.6:
            detailed_label = 'Strong Positive Opinion'
        elif 0.1 < polarity <= 0.5 and subjectivity > 0.6:
            detailed_label = 'Mild Positive Opinion'
        elif polarity > 0.5 and subjectivity < 0.4:
            detailed_label = 'Positive Fact'
        elif -0.1 <= polarity <= 0.1 and subjectivity > 0.6:
            detailed_label = 'Neutral Opinion'
        elif -0.1 <= polarity <= 0.1 and subjectivity < 0.4:
            detailed_label = 'Neutral Fact'
        elif -0.5 < polarity <= -0.1 and subjectivity > 0.6:
            detailed_label = 'Mild Negative Opinion'
        elif polarity < -0.5 and subjectivity > 0.6:
            detailed_label = 'Strong Negative Opinion'
        elif polarity < -0.5 and subjectivity < 0.4:
            detailed_label = 'Negative Fact'
        else:
            detailed_label = 'Negative Fact'

        # Mapping detailed categories to broader Positive, Neutral, Negative
        if 'Positive' in detailed_label:
            return 'Positive'
        elif 'Negative' in detailed_label:
            return 'Negative'
        elif 'Neutral' in detailed_label:
            return 'Neutral'
        else:
            return 'Unclassified'

    df['Label'] = df.apply(label_sentiment, axis=1)
    return df

In [26]:
labeled_df = label_sentiments_in_dataframe(df, 'sentiment')
labeled_df = labeled_df[['clean_text','Label' ]]

IndexError: list index out of range